# 프로젝트 - Movielens 영화 추천 실습
## <목차>
#### STEP1. 데이터 준비와 전처리
#### STEP2. 데이터 준비와 전처리
#### 🤔회고🤔
#### Reference


# STEP1. 데이터 준비와 전처리

In [1]:
import pandas as pd
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [4]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
# 검색을 쉽게 하기 위해 영화 제목을 소문자로 변경
movies['title'] = movies['title'].str.lower() # 검색을 쉽게 하기 위해 아티스트 문자열을 소문자로 바꿔줍시다.
movies.head(10)

,movie_id,title,genre
0,1,toy story (1995),Animation|Children's|Comedy
1,2,jumanji (1995),Adventure|Children's|Fantasy
2,3,grumpier old men (1995),Comedy|Romance
3,4,waiting to exhale (1995),Comedy|Drama
4,5,father of the bride part ii (1995),Comedy
5,6,heat (1995),Action|Crime|Thriller
6,7,sabrina (1995),Comedy|Romance
7,8,tom and huck (1995),Adventure|Children's
8,9,sudden death (1995),Action
9,10,goldeneye (1995),Action|Adventure|Thriller


In [6]:
type(movies)

pandas.core.frame.DataFrame

- 내가 좋아하는 영화 지정을 위해 전체 영화 리스트 뽑아보기
- 영화들이 너무 예전 영화들이라 그냥 랜덤으로 5개 뽑고자 한다..

In [7]:
title_list = movies['title'].values.tolist()
print(title_list)

['toy story (1995)', 'jumanji (1995)', 'grumpier old men (1995)', 'waiting to exhale (1995)', 'father of the bride part ii (1995)', 'heat (1995)', 'sabrina (1995)', 'tom and huck (1995)', 'sudden death (1995)', 'goldeneye (1995)', 'american president, the (1995)', 'dracula: dead and loving it (1995)', 'balto (1995)', 'nixon (1995)', 'cutthroat island (1995)', 'casino (1995)', 'sense and sensibility (1995)', 'four rooms (1995)', 'ace ventura: when nature calls (1995)', 'money train (1995)', 'get shorty (1995)', 'copycat (1995)', 'assassins (1995)', 'powder (1995)', 'leaving las vegas (1995)', 'othello (1995)', 'now and then (1995)', 'persuasion (1995)', 'city of lost children, the (1995)', 'shanghai triad (yao a yao yao dao waipo qiao) (1995)', 'dangerous minds (1995)', 'twelve monkeys (1995)', 'wings of courage (1995)', 'babe (1995)', 'carrington (1995)', 'dead man walking (1995)', 'across the sea of time (1995)', 'it takes two (1995)', 'clueless (1995)', 'cry, the beloved country (199

In [18]:
import random
random_title = random.sample(title_list, 5)
random_title

['still breathing (1997)',
 'i love trouble (1994)',
 'golden voyage of sinbad, the (1974)',
 'rain (1932)',
 'children of the corn ii: the final sacrifice (1993)']

In [33]:
#random_title_5 = [''still breathing (1997)','i love trouble (1994)','golden voyage of sinbad, the (1974)','rain (1932)','children of the corn ii: the final sacrifice (1993)']
movies[movies['title'].isin(random_title)]

,movie_id,title,genre
356,360,i love trouble (1994),Action|Comedy
1805,1874,still breathing (1997),Comedy|Romance
2446,2515,children of the corn ii: the final sacrifice (...,Horror
2835,2904,rain (1932),Drama
3702,3771,"golden voyage of sinbad, the (1974)",Action|Adventure


In [8]:
ratings.loc[ratings['user_id'] == 1]

,user_id,movie_id,counts,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
5,1,1197,3,978302268
6,1,1287,5,978302039
7,1,2804,5,978300719
8,1,594,4,978302268
9,1,919,4,978301368


In [9]:
merge_movies = pd.merge(ratings, movies, how='left', on='movie_id')
#merge_movies.sort_values('user_id')
merge_movies.loc[merge_movies['user_id'] == 1]

,user_id,movie_id,counts,timestamp,title,genre
0,1,1193,5,978300760,one flew over the cuckoo's nest (1975),Drama
1,1,661,3,978302109,james and the giant peach (1996),Animation|Children's|Musical
2,1,914,3,978301968,my fair lady (1964),Musical|Romance
3,1,3408,4,978300275,erin brockovich (2000),Drama
4,1,2355,5,978824291,"bug's life, a (1998)",Animation|Children's|Comedy
5,1,1197,3,978302268,"princess bride, the (1987)",Action|Adventure|Comedy|Romance
6,1,1287,5,978302039,ben-hur (1959),Action|Adventure|Drama
7,1,2804,5,978300719,"christmas story, a (1983)",Comedy|Drama
8,1,594,4,978302268,snow white and the seven dwarfs (1937),Animation|Children's|Musical
9,1,919,4,978301368,"wizard of oz, the (1939)",Adventure|Children's|Drama|Musical


In [10]:
print(ratings.shape[0])
print(merge_movies.shape[0])

836478
836478


### 필요없는 데이터 열 제거하기

In [11]:
# 필요없는 데이터 열 제거
merge_movies_1 = merge_movies.drop(['timestamp', 'genre'], axis=1)

# 컬럼명 변경하기
merge_movies_1.rename(columns={'ratings':'counts'}, inplace=True)

merge_movies_1.tail()

,user_id,movie_id,counts,title
836473,6040,1090,3,platoon (1986)
836474,6040,1094,5,"crying game, the (1992)"
836475,6040,562,5,welcome to the dollhouse (1995)
836476,6040,1096,4,sophie's choice (1982)
836477,6040,1097,4,e.t. the extra-terrestrial (1982)


### 모델 검증을 위한 사용자 초기 정보 세팅
- 사용자 초기 정보 세팅
- 마지막 user_id는 6040임을 확인

In [12]:
my_favorite_id = [360, 1874, 2515, 2904, 3771]
my_favorite_title = ['i love trouble (1994)', 'still breathing (1997)', 'children of the corn ii: the final sacrifice (1993)', 'rain (1932)','golden voyage of sinbad, the (1974)']
my_favorite_count	 = [5,4,3,5,5]

my_movielist = pd.DataFrame({'user_id': ['gabi']*5, 'movie_id': my_favorite_id, 'counts': my_favorite_count, 'title': my_favorite_title})
my_movielist

,user_id,movie_id,counts,title
0,gabi,360,5,i love trouble (1994)
1,gabi,1874,4,still breathing (1997)
2,gabi,2515,3,children of the corn ii: the final sacrifice (...
3,gabi,2904,5,rain (1932)
4,gabi,3771,5,"golden voyage of sinbad, the (1974)"


In [13]:
merge_movies_1 = merge_movies_1.append(my_movielist, ignore_index=True)
merge_movies_1.tail(10)

,user_id,movie_id,counts,title
836473,6040,1090,3,platoon (1986)
836474,6040,1094,5,"crying game, the (1992)"
836475,6040,562,5,welcome to the dollhouse (1995)
836476,6040,1096,4,sophie's choice (1982)
836477,6040,1097,4,e.t. the extra-terrestrial (1982)
836478,gabi,360,5,i love trouble (1994)
836479,gabi,1874,4,still breathing (1997)
836480,gabi,2515,3,children of the corn ii: the final sacrifice (...
836481,gabi,2904,5,rain (1932)
836482,gabi,3771,5,"golden voyage of sinbad, the (1974)"


## 모델에 활용하기 위한 전처리
-  데이터의 관리를 쉽게 하기 위해 번호 붙이기
- pandas.DataFrame.unique() : 특정 컬럼에 포함된 유니크한 데이터만 모아 줌   
    => indexing작업 할 때 매우 유용

In [14]:
merge_movies_1['user_id'].unique()

array([1, 2, 3, ..., 6039, 6040, 'gabi'], dtype=object)

In [15]:
# 고유한 유저, 제목을 찾아내는 코드
user_unique = merge_movies_1['user_id'].unique()
title_unique = merge_movies_1['title'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
title_to_idx = {v:k for k,v in enumerate(title_unique)}

# 인덱싱 확인하기
#print(user_to_idx['gabi']) 
#print(title_to_idx['rain (1932)'])

In [16]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = merge_movies_1['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(merge_movies_1):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    merge_movies_1['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_title_data = merge_movies_1['title'].map(title_to_idx.get).dropna()
if len(temp_title_data) == len(merge_movies_1):
    print('title column indexing OK!!')
    merge_movies_1['title'] = temp_title_data
else:
    print('title column indexing Fail!!')

merge_movies_1

user_id column indexing OK!!
title column indexing OK!!


,user_id,movie_id,counts,title
0,0,1193,5,0
1,0,661,3,1
2,0,914,3,2
3,0,3408,4,3
4,0,2355,5,4
...,...,...,...,...
836478,6039,360,5,2075
836479,6039,1874,4,2671
836480,6039,2515,3,3099
836481,6039,2904,5,2099


## 명시적/암묵적 평가
- 평가 기준에 대해서 어떻게 설정할지 고민이 필요함

# STEP2. 분석하기
- ratings에 있는 유니크한 영화 개수
- ratings에 있는 유니크한 사용자 수
- 가장 인기 있는 영화 30개(인기순)

In [17]:
ratings['movie_id'].nunique()

3628

In [18]:
ratings['user_id'].nunique()

6039

In [19]:
merge_movies_1['user_id'].nunique()

6040

In [20]:
movie_count = ratings.groupby('movie_id')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: user_id, dtype: int64

# STEP3.CSR matrix

In [21]:
# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

num_user = merge_movies_1['user_id'].nunique()
num_title = merge_movies_1['title'].nunique()

csr_data = csr_matrix((merge_movies_1['counts'], (merge_movies_1.user_id, merge_movies_1.title)), shape= (num_user, num_title))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

# STEP4. 모델 훈련
-  als_model = AlternatingLeastSquares 모델 구성
- 파라미터
    1. factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지
    2. regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지
    3. use_gpu : GPU를 사용할 것인지
    4. iterations : epochs와 같은 의미입니다. 데이터를 몇 번 반복해서 학습할 것인지
- factors와 iterations를 늘릴수록 학습능률이 좋아지지만 과적합 우려도 있음

In [22]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [23]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [24]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [25]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

# STEP5. 선호도 파악!

In [26]:
gabi, matrix = user_to_idx['gabi'], title_to_idx['rain (1932)']
gabi_vector, matrix_vector = als_model.user_factors[gabi], als_model.item_factors[matrix]

In [27]:
gabi_vector

array([ 0.20133694, -0.09016997, -0.06060972, -0.02012986,  0.0656352 ,
        0.10970166, -0.08280108,  0.15182775,  0.15372151,  0.17834124,
       -0.1748787 , -0.28717357,  0.15334164,  0.09840963, -0.03209021,
       -0.06112405, -0.08933266, -0.10035142, -0.3540044 , -0.13579524,
       -0.29563668, -0.23067714,  0.17741035, -0.28287625,  0.04527569,
        0.29123354, -0.05808001, -0.04643271, -0.01372215, -0.26766863,
        0.12165932,  0.14547567, -0.01017731, -0.13430811,  0.35704944,
       -0.04174675,  0.04088593, -0.03839439, -0.14019777, -0.11020277,
       -0.23113702,  0.06989498, -0.11280386,  0.10073666,  0.09100914,
        0.02674768,  0.1177223 ,  0.05750098,  0.09048765, -0.04334283,
        0.3717749 ,  0.00742569, -0.07285551,  0.10434594,  0.03396563,
        0.07588374,  0.1099987 , -0.14705692, -0.33689997,  0.23479235,
       -0.14017722, -0.0877366 ,  0.42008972,  0.1348534 , -0.31423536,
       -0.02451784, -0.20009238, -0.28077924,  0.5505127 , -0.18

In [28]:
matrix_vector

array([ 4.5019188e-03, -2.2326035e-03,  2.2192809e-03,  2.4407767e-03,
       -4.2014248e-03,  3.9555705e-03,  3.8119436e-03,  4.7130338e-03,
        3.5788627e-05,  3.9005014e-03,  1.7309106e-03,  2.6735656e-03,
        6.8301610e-03,  3.7667921e-03,  9.7137606e-03,  5.5342223e-03,
       -1.0106656e-03,  8.0537731e-03, -9.0702286e-04,  3.0168924e-03,
       -2.8740230e-04,  4.6236529e-03,  6.6321893e-03, -8.1300089e-04,
        4.8495308e-03,  9.3919067e-03,  5.1603030e-04,  2.7649878e-03,
        2.5005506e-03,  6.7916005e-03,  4.2741862e-03,  4.7474708e-03,
        9.3648005e-03,  4.8605418e-03,  1.0092778e-03,  3.0883686e-03,
        4.4374590e-04,  3.1811958e-03,  1.7757972e-03, -2.0022930e-04,
        9.0984901e-04,  4.8360396e-03,  3.6116242e-03,  8.5499493e-04,
        4.5139487e-03,  3.8363899e-03,  3.9128922e-03,  2.6724755e-03,
       -3.6481419e-03,  4.7808755e-03,  1.4922918e-03,  1.8963051e-03,
        4.6747113e-03,  3.3600037e-03, -9.1231021e-04,  3.9571109e-03,
      

In [29]:
# zimin과 black_eyed_peas를 내적하는 코드
np.dot(gabi_vector, matrix_vector)

0.017964568

# STEP6. 비슷한 영화 추천
## (1) 비슷한 영화 찾기

In [30]:
favorite_movie = 'rain (1932)'
title_id = title_to_idx[favorite_movie]
similar_movie = als_model.similar_items(title_id, N=15)
similar_movie

[(2099, 0.9999999),
 (3585, 0.8778389),
 (3554, 0.8605128),
 (3492, 0.85194445),
 (2521, 0.85012263),
 (3539, 0.84996325),
 (3576, 0.8396465),
 (3574, 0.8396117),
 (3575, 0.8395522),
 (3572, 0.8391708),
 (3579, 0.83910114),
 (3583, 0.8390721),
 (3573, 0.8384305),
 (3580, 0.83842283),
 (3578, 0.8382856)]

In [31]:
#artist_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성합니다. 
idx_to_title = {v:k for k,v in title_to_idx.items()}
[idx_to_title[i[0]] for i in similar_movie]

['rain (1932)',
 'macao (1952)',
 'tomb of ligeia, the (1965)',
 'last time i saw paris, the (1954)',
 'life of émile zola, the (1937)',
 "farmer's wife, the (1928)",
 'war at home, the (1996)',
 'century (1993)',
 "brother's kiss, a (1997)",
 "i don't want to talk about it (de eso no se habla) (1993)",
 'last of the high kings, the (a.k.a. summer fling) (1996)',
 'neon bible, the (1995)',
 'male and female (1919)',
 "another man's poison (1952)",
 'number seventeen (1932)']

- 위 과정을 간단하게 함수로 나타내기!

In [32]:
def get_similar_movie(title_name: str):
    title_id = title_to_idx[title_name]
    similar_movie = als_model.similar_items(title_id)
    similar_movie = [idx_to_title[i[0]] for i in similar_movie]
    return similar_movie

In [33]:
get_similar_movie('rain (1932)')

['rain (1932)',
 'macao (1952)',
 'tomb of ligeia, the (1965)',
 'last time i saw paris, the (1954)',
 'life of émile zola, the (1937)',
 "farmer's wife, the (1928)",
 'war at home, the (1996)',
 'century (1993)',
 "brother's kiss, a (1997)",
 "i don't want to talk about it (de eso no se habla) (1993)"]

## (2) 내가 좋아할 만한 영화 추천
- filter_already_liked_items : 이미 평가한 아이템 제외

In [34]:
user = user_to_idx['gabi']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(1541, 0.090957165),
 (1544, 0.07496716),
 (224, 0.07231678),
 (1405, 0.06518006),
 (492, 0.06358318),
 (677, 0.061947025),
 (1309, 0.0604011),
 (1307, 0.058594704),
 (500, 0.05734362),
 (1131, 0.057105556),
 (1106, 0.05651152),
 (156, 0.05596886),
 (119, 0.055388916),
 (164, 0.053792026),
 (1137, 0.053577237),
 (176, 0.053505763),
 (515, 0.052746296),
 (2104, 0.052731626),
 (999, 0.052173935),
 (67, 0.051944684)]

In [35]:
[idx_to_title[i[0]] for i in movie_recommended]

['7th voyage of sinbad, the (1958)',
 'sinbad and the eye of the tiger (1977)',
 'l.a. confidential (1997)',
 'kentucky fried movie, the (1977)',
 "midsummer night's dream, a (1999)",
 'army of darkness (1993)',
 "kelly's heroes (1970)",
 'godzilla (gojira) (1954)',
 'chinatown (1974)',
 'big trouble in little china (1986)',
 'midnight run (1988)',
 'mask of zorro, the (1998)',
 'twister (1996)',
 '28 days (2000)',
 'dreamscape (1984)',
 'ghost and the darkness, the (1996)',
 'batman forever (1995)',
 'king kong (1976)',
 'evil dead ii (dead by dawn) (1987)',
 'gladiator (2000)']

In [36]:
why = title_to_idx['7th voyage of sinbad, the (1958)']
explain = als_model.explain(user, csr_data, itemid=why)

[(idx_to_title[i[0]], i[1]) for i in explain[1]]

[('golden voyage of sinbad, the (1974)', 0.08851433016437883),
 ('rain (1932)', 0.004784472291167673),
 ('children of the corn ii: the final sacrifice (1993)',
  0.0018207406033211033),
 ('still breathing (1997)', -0.0002723510794199142),
 ('i love trouble (1994)', -0.005730243598987585)]

# 🤔회고🤔
### <루브릭 평가 지표>

|평가문항|상세기준|
|---|---|
|1. CSR matrix가 정상적으로 만들어졌다.|사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들었다.|
|2. MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다.|사용자와 아이템 벡터 내적수치가 의미있게 형성되었다.|
|3. 비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다.|MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도가 의미있게 측정되었다.|

***
### <이번 프로젝트에서 **어려웠던 점**>
- 실제로 내가 좋아하는 'life of pi', 'the lovely bones', 'the white tiger' 등의 영화는 리스트에 없어서 너무 아쉬웠다 ㅠㅠ 실제로 추천되는 영화를 보려고 했건만..!
- 리스트 영화들이 너무 예전 영화들이라 대충 암거나 골라잡아야하는데 상당히 아쉽,,,ㅠㅠ
- 제일 관심있는 분야였기 때문에 처음부터 하나하나 이해하려고 하니 딜레이가 심했고, 들인 시간에 비해 비해가 부족한 것 같아 어려웠음


### <프로젝트를 진행하면서 **알아낸 점** 혹은 **아직 모호한 점**>

### <루브릭 평가 지표를 맞추기 위해 **시도한 것들**>


### <만약에 루브릭 평가 관련 지표를 **달성 하지 못했을 때, 이유에 관한 추정**>


### < **자기 다짐**>
 원하는 구현 방식이 있는데 계속해서 시도해봐야겠다.


# Reference
1. df에서 list로 변환 : https://ddolcat.tistory.com/729
2. list에서 랜덤으로 값 뽑기 : https://enfanthoon.tistory.com/108
3. df에서 특정 값 추출 : https://computer-science-student.tistory.com/375